In [1]:
import os
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
openai_key = os.getenv("OPENAI_API_KEY")

In [3]:
import instructor

from openai import OpenAI
from typing import List
from pydantic import BaseModel, Field

client = instructor.patch(OpenAI())

In [4]:
from datetime import date


class DateRange(BaseModel):
    start: date
    end: date


class Query(BaseModel):
    rewritten_query: str
    published_daterange: DateRange

In [5]:
def expand_query(q) -> Query:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=Query,
        messages=[
            {
                "role": "system",
                "content": f"You are a ",
            },
            {"role": "user", "content": f"query: {q}"},
        ],
    )


query = expand_query("What are some recent developments in AI?")
query

Query(rewritten_query='Recent developments in artificial intelligence', published_daterange=DateRange(start=datetime.date(2022, 1, 1), end=datetime.date(2022, 4, 30)))

In [6]:
class DateRange(BaseModel):
    chain_of_thought: str = Field(
        description="Think step by step to plan what is the best time range to search in"
    )
    start: date
    end: date

class Tools(BaseModel):
    tool: str = Field(description="Tool to use for the search")
    description: str = Field(description="Description of the tool")
    link: str = Field(description="Link to the tool")

class Query(BaseModel):
    rewritten_query: str = Field(
        description="Rewrite the query to make it more specific"
    )
    published_daterange: DateRange = Field(
        description="Effective date range to search in"
    )


def expand_query(q) -> Query:
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=Query,
        messages=[
            {
                "role": "system",
                "content": f"You're a query understanding system for the Metafor Systems search engine. Here are some tips: ...",
            },
            {"role": "user", "content": f"query: {q}"},
        ],
    )


expand_query("What are some recent developments in AI?")

Query(rewritten_query='latest advancements in artificial intelligence 2023', published_daterange=DateRange(chain_of_thought="Since it's 2023 and the user is asking about recent developments, focusing on AI advancements from the beginning of this year to the present would be ideal to ensure the information is up-to-date.", start=datetime.date(2023, 1, 1), end=datetime.date(2023, 4, 30)))

In [7]:
querystring="What are some recent developments in AI?"

retrival=client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=Query,
        messages=[
            {
                "role": "system",
                "content": f"You're a query understanding system for the Metafor Systems search engine. Today is {date.today()}. Here are some tips: ...",
            },
            {"role": "user", "content": f"query: {querystring}"},
        ],
    )
print(retrival.model_dump_json(indent=4))

{
    "rewritten_query": "latest advancements in artificial intelligence",
    "published_daterange": {
        "chain_of_thought": "Since it's currently 2024 and 'recent' typically refers to developments that have occurred over the past year or so, the date range for the search could start from early 2023 to the present date in 2024.",
        "start": "2023-01-01",
        "end": "2024-05-12"
    }
}


In [26]:
class Endpoints(BaseModel):
    id: int = Field(..., description="A unique identifier for the question")
    query: str = Field(..., description="The question decomposited as much as possible")
    endpoints: List[str] = Field(..., description="The graphql endpoint that is required to fetch data from the ai to answer this specific question")
    arguments: List[str] = Field(..., description="The parameters that are required to send in the query request to the endpoint")
    subquestions: List[int] = Field(
        default_factory=list,
        description="The other endpoints it relies on to get the data from to answer the question. Check each of the arguments of the endpoint to check if it is available if not add a subquestion to get the data from the other point that it can be retrived from",
    )


class QueryPlan(BaseModel):
    root_question: str = Field(..., description="The root question that the user asked")
    plan: List[Endpoints] = Field(
        ..., description="""The plan to answer the root question by querying different endpoints available in the graphql api
        Make sure every information is present and to answer the question and decompose the question properly into each of it's respective endpoints"""
    )


retrival = client.chat.completions.create(
    model="gpt-4-turbo",
    response_model=QueryPlan,
    messages=[
        {
            "role": "system",
            "content": '''You are a query understanding system capable of decomposing a question into subparamaters required to answer the question.
            
            The arguments which are present in different parts of the graphql endpoint i am about to use are as follows: 
           These are the endpoints that are available in graphql
           type Query {
  searchVesselsByIdentifier(identifier: String!, limit: PositiveInt = 20): [SimplifiedVessel]

  """Query - returns information on a single vessel behaviour"""
  vessel(id: ObjectId!): VesselIntelligence

  """Query - returns information on a single vessel by its IMO"""
  vesselByIMO(imo: String!): VesselIntelligence

  """Query - returns information on a multiple vessels by their IMO"""
  vesselsByIMOs(imos: [String!]!): [VesselIntelligence]
  vesselsByMMSI(mmsi: String!): [VesselIntelligence!]!
  portExpectedArrivals(input: PortExpectedArrivalsInput!): PortExpectedArrivalsConnection!
  vesselsInPort(input: VesselsInPortInput!): VesselsInPortConnection!
  departedFromPortVessels(input: DepartedFromPortVesselsInput!): DepartedFromPortVesselsConnection!
  vesselPropertyChanges(input: VesselPropertyChangesInput!): VesselPropertyChangesConnection!

  """
  The API requires selection of a user defined area and selected time range 
  and returns a list of vessels that transmitted in that selected location and selected time
  """
  vesselsInArea(input: VesselsInAreaInput!): VesselsInAreaConnection!
  vesselsCurrentlyInArea(input: VesselsCurrentlyInAreaInput!): VesselsCurrentlyInAreaConnection!
  riskyVesselsInArea(input: RiskyVesselsInAreaInput!): RiskyVesselsInAreaConnection!
  getActivitiesByDatesAndPolygon(type: ActivityTypes!, timeRange: DateTimeRange!, polygonId: ObjectId!): [Activity]
  activitiesInPolygon(input: ActivitiesInPolygonInput!): ActivitiesInPolygonConnection!
  vesselTimeline(input: VesselTimelineInput!): VesselTimelineConnection!
  advancedVesselsSearch(input: AdvancedVesselSearchInput!): SearchResultsOutput
  areas(filter: AreaFilterInput!): [FeatureObject]
  searchCompaniesByTerm(searchTerm: String!): [Company!]!
  projectVOIs: [VOI]
  voiAuditLogs(input: VOIAuditLogsInput!): VOIAuditLogsConnection!
  complianceServiceReport(imos: [String], timeRange: DateTimeRange, programs: [ComplianceProgram], risks: [Int]): ComplianceServiceReport
  complianceRiskBy(input: ComplianceRiskByInput!): ComplianceRiskByConnection!
  complianceVesselBuildingBlocksBy(input: ComplianceVesselBuildingBlocksByInput!): ComplianceVesselBuildingBlocksByConnection!

            ''',

        },
        {
            "role": "user",
            "content": "what are the ships surrounding india and what of them has imo=9987654",
        },
    ],
)

print(retrival.model_dump_json(indent=4))

{
    "root_question": "What are the ships surrounding India and which of them has IMO=9987654?",
    "plan": [
        {
            "id": 1,
            "query": "Get details of ships surrounding India",
            "endpoints": [
                "vesselsInArea"
            ],
            "arguments": [
                "input"
            ],
            "subquestions": [
                3
            ]
        },
        {
            "id": 2,
            "query": "Fetch details of a specific vessel by IMO number 9987654",
            "endpoints": [
                "vesselByIMO"
            ],
            "arguments": [
                "imo"
            ],
            "subquestions": []
        },
        {
            "id": 3,
            "query": "Define area polygon surrounding India",
            "endpoints": [
                "areas"
            ],
            "arguments": [
                "filter"
            ],
            "subquestions": []
        }
    ]
}
